<a href="https://colab.research.google.com/github/Yassmina-Abdo/pyspark_course/blob/main/Lab1_Practical_Session_1_G2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Lab 1 PySpark:** 


In these labs we will be using the "[[NeurIPS 2020] Data Science for COVID-19 (DS4C)](https://www.kaggle.com/datasets/kimjihoo/coronavirusdataset?select=PatientInfo.csv)" dataset, retrieved from [Kaggle](https://www.kaggle.com/) on 1/6/2022, for educational non commercial purpose, License
[CC BY-NC-SA 4.0
](https://creativecommons.org/licenses/by-nc-sa/4.0/)


The csv files that we will be using in this lab is **PatientInfo**.



# File 📁

##PatientInfo.csv

**patient_id**
the ID of the patient

**sex**
the sex of the patient

**age**
the age of the patient

**country**
the country of the patient

**province**
the province of the patient

**city**
the city of the patient

**infection_case**
the case of infection

**infected_by**
the ID of who infected the patient


**contact_number**
the number of contacts with people

**symptom_onset_date**
the date of symptom onset

**confirmed_date**
the date of being confirmed

**released_date**
the date of being released

**deceased_date**
the date of being deceased

**state**
isolated / released / deceased

# Installing PySpark:


Kindly install pyspark ⌨

In [1]:
!pip install pyspark

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 281.4 MB 33 kB/s 
     |████████████████████████████████| 198 kB 54.5 MB/s 
  Created wheel for pyspark: filename=pyspark-3.2.1-py2.py3-none-any.whl size=281853642 sha256=bce29feb84d167e5c898171ea8ad42916fc61098f4173ca96b845c8518d22da5
  Stored in directory: /root/.cache/pip/wheels/9f/f5/07/7cd8017084dce4e93e84e92efd1e1d5334db05f2e83bcef74f
Successfully built pyspark


Check the pyspark version

In [2]:
import pyspark
from pyspark.sql import SparkSession
spark = SparkSession.builder.master("local[1]").appName('SparkByExamples.com').getOrCreate()

print('PySpark Version :'+spark.version)

PySpark Version :3.2.1


Importing sparksession then create a spark session

In [3]:
import pyspark
from pyspark.sql import SparkSession

In [4]:
spark = SparkSession.builder.appName('Lab1_G2').getOrCreate()

# Follow the instructions above the cells below:


Kindly load the PatientInfo.csv file and show the first 5 rows

❌ PLEASE do NOT change the cell below 

In [5]:
!gdown https://drive.google.com/uc?id=1PB6wBDVTM_eocxOyi0lWlLBQOlH0rLe_ -O PatientInfo.csv

Downloading...
From: https://drive.google.com/uc?id=1PB6wBDVTM_eocxOyi0lWlLBQOlH0rLe_
To: /content/PatientInfo.csv
100% 489k/489k [00:00<00:00, 117MB/s]


load the **PatientInfo.csv** file downloaded and put it in a dataframe called **patient**

In [6]:
df = spark.read.csv('/content/PatientInfo.csv', header = True, inferSchema=True)

Show the first 5 rows

In [7]:
df.show(5)

+----------+------+---+-------+--------+-----------+--------------------+-----------+--------------+------------------+--------------+-------------+-------------+--------+
|patient_id|   sex|age|country|province|       city|      infection_case|infected_by|contact_number|symptom_onset_date|confirmed_date|released_date|deceased_date|   state|
+----------+------+---+-------+--------+-----------+--------------------+-----------+--------------+------------------+--------------+-------------+-------------+--------+
|1000000001|  male|50s|  Korea|   Seoul| Gangseo-gu|     overseas inflow|       null|            75|        2020-01-22|    2020-01-23|   2020-02-05|         null|released|
|1000000002|  male|30s|  Korea|   Seoul|Jungnang-gu|     overseas inflow|       null|            31|              null|    2020-01-30|   2020-03-02|         null|released|
|1000000003|  male|50s|  Korea|   Seoul|  Jongno-gu|contact with patient| 2002000001|            17|              null|    2020-01-30|   202

Display the schema for the dataset

In [8]:
df.printSchema()

root
 |-- patient_id: long (nullable = true)
 |-- sex: string (nullable = true)
 |-- age: string (nullable = true)
 |-- country: string (nullable = true)
 |-- province: string (nullable = true)
 |-- city: string (nullable = true)
 |-- infection_case: string (nullable = true)
 |-- infected_by: string (nullable = true)
 |-- contact_number: string (nullable = true)
 |-- symptom_onset_date: string (nullable = true)
 |-- confirmed_date: string (nullable = true)
 |-- released_date: string (nullable = true)
 |-- deceased_date: string (nullable = true)
 |-- state: string (nullable = true)



Display the statistical summary

In [9]:
df.summary().show()

+-------+--------------------+------+----+----------+--------+--------------+--------------------+--------------------+--------------------+------------------+--------------+-------------+-------------+--------+
|summary|          patient_id|   sex| age|   country|province|          city|      infection_case|         infected_by|      contact_number|symptom_onset_date|confirmed_date|released_date|deceased_date|   state|
+-------+--------------------+------+----+----------+--------+--------------+--------------------+--------------------+--------------------+------------------+--------------+-------------+-------------+--------+
|  count|                5165|  4043|3785|      5165|    5165|          5071|                4246|                1346|                 791|               690|          5162|         1587|           66|    5165|
|   mean|2.8636345618679576E9|  null|null|      null|    null|          null|                null|2.2845944015643125E9|1.6772572523506988E7|            

How many people survived, and how many didn't survive?

In [10]:
not_survived = df.filter(df.state == 'deceased').count()
survived = (df.filter(df.state == 'released').count()) + (df.filter(df.state == 'isolated').count())
print(not_survived)
print(survived)

78
5087


Display the number of null values in each column

In [11]:
from pyspark.sql.functions import col,isnan, when, count
df.select([count(when(isnan(c) | col(c).isNull(), c)).alias(c) for c in df.columns]).show()

+----------+----+----+-------+--------+----+--------------+-----------+--------------+------------------+--------------+-------------+-------------+-----+
|patient_id| sex| age|country|province|city|infection_case|infected_by|contact_number|symptom_onset_date|confirmed_date|released_date|deceased_date|state|
+----------+----+----+-------+--------+----+--------------+-----------+--------------+------------------+--------------+-------------+-------------+-----+
|         0|1122|1380|      0|       0|  94|           919|       3819|          4374|              4475|             3|         3578|         5099|    0|
+----------+----+----+-------+--------+----+--------------+-----------+--------------+------------------+--------------+-------------+-------------+-----+



# Data preprocessing

Kindly fill the nulls in the **deceased_date** with the **released_date**

In [12]:
from pyspark.sql.functions import coalesce

Drop values having a null deceased_date

In [13]:
df = df.na.drop(subset="deceased_date")
df.show()

+----------+------+---+-------+--------+------+------------------+-----------+--------------+------------------+--------------+-------------+-------------+--------+
|patient_id|   sex|age|country|province|  city|    infection_case|infected_by|contact_number|symptom_onset_date|confirmed_date|released_date|deceased_date|   state|
+----------+------+---+-------+--------+------+------------------+-----------+--------------+------------------+--------------+-------------+-------------+--------+
|1200000038|female|50s|  Korea|   Daegu|Nam-gu|              null|       null|          null|              null|    2020-02-18|         null|   2020-02-23|deceased|
|1200000114|  male|70s|  Korea|   Daegu|  null|Shincheonji Church|       null|          null|              null|    2020-02-21|         null|   2020-02-26|deceased|
|1200000620|  male|70s|  Korea|   Daegu|  null|              null|       null|          null|              null|    2020-02-24|         null|   2020-03-02|deceased|
|120000090

Add a column named no_days which is difference between the deceased_date and the confirmed_date then show the top 5 rows

Hint: You need to typecase these coumns as date first

In [14]:
from pyspark.sql.functions import datediff, to_date

df = df.withColumn("no_days", datediff(to_date(df['deceased_date']) , to_date(df['confirmed_date'])))
df.select('no_days').show(5)

+-------+
|no_days|
+-------+
|      5|
|      5|
|      7|
|      8|
|      4|
+-------+
only showing top 5 rows



In [15]:
df.show(5)

+----------+------+---+-------+--------+------+------------------+-----------+--------------+------------------+--------------+-------------+-------------+--------+-------+
|patient_id|   sex|age|country|province|  city|    infection_case|infected_by|contact_number|symptom_onset_date|confirmed_date|released_date|deceased_date|   state|no_days|
+----------+------+---+-------+--------+------+------------------+-----------+--------------+------------------+--------------+-------------+-------------+--------+-------+
|1200000038|female|50s|  Korea|   Daegu|Nam-gu|              null|       null|          null|              null|    2020-02-18|         null|   2020-02-23|deceased|      5|
|1200000114|  male|70s|  Korea|   Daegu|  null|Shincheonji Church|       null|          null|              null|    2020-02-21|         null|   2020-02-26|deceased|      5|
|1200000620|  male|70s|  Korea|   Daegu|  null|              null|       null|          null|              null|    2020-02-24|        

Add a **is_male** column if male then it should yield 1, else then 0 and show your dataframe

In [16]:
from pyspark.sql.functions import when, lit
df = df.withColumn("is_male", \
   when((df.sex == 'male'), lit(1))\
     .otherwise(lit(0)) \
  )
df.select(['sex', 'is_male']).show()

+------+-------+
|   sex|is_male|
+------+-------+
|female|      0|
|  male|      1|
|  male|      1|
|female|      0|
|female|      0|
|  male|      1|
|  male|      1|
|  male|      1|
|  male|      1|
|  male|      1|
|female|      0|
|female|      0|
|  male|      1|
|female|      0|
|  male|      1|
|  male|      1|
|  male|      1|
|female|      0|
|  male|      1|
|  male|      1|
+------+-------+
only showing top 20 rows



Add a **is_dead** column if patient state is released then make it 1, else 0 and show your dataframe

In [17]:
from pyspark.sql.functions import when, lit
df = df.withColumn("is_dead", \
   when((df.state == 'deceased'), lit(1))\
     .otherwise(lit(0)) \
  )
df.select(['state', 'is_dead']).show()

+--------+-------+
|   state|is_dead|
+--------+-------+
|deceased|      1|
|deceased|      1|
|deceased|      1|
|deceased|      1|
|deceased|      1|
|deceased|      1|
|deceased|      1|
|deceased|      1|
|deceased|      1|
|deceased|      1|
|deceased|      1|
|deceased|      1|
|deceased|      1|
|deceased|      1|
|deceased|      1|
|deceased|      1|
|deceased|      1|
|deceased|      1|
|deceased|      1|
|deceased|      1|
+--------+-------+
only showing top 20 rows



Kindly change the ages to bins from 10s, 0s, 10s, 20s,.etc to 0,10, 20 and show your dataframe

In [18]:
from pyspark.sql.types import IntegerType
import pyspark.sql.functions as f

#normal funvtion
def remove_s(age):
  if age:
    new_age = int(age[:-1])
    return new_age
  else:
    return "None"

remove_S_udf = f.udf(lambda x : remove_s(x), IntegerType())
spark.udf.register('remove_S_udf', remove_S_udf)

<function __main__.<lambda>>

In [19]:
df = df.withColumn("age",remove_S_udf('age'))
df.select('age').show()

+---+
|age|
+---+
| 50|
| 70|
| 70|
| 80|
| 70|
| 60|
| 70|
| 60|
| 70|
| 60|
| 70|
| 80|
| 80|
| 60|
| 90|
| 80|
| 80|
| 50|
| 80|
| 70|
+---+
only showing top 20 rows



Repeat the same, but for the **no_days** return the numbers only and show your dataframe

In [20]:
df = df.withColumn("no_days",df.no_days.cast('double'))

Print the schema

In [21]:
df.printSchema()

root
 |-- patient_id: long (nullable = true)
 |-- sex: string (nullable = true)
 |-- age: integer (nullable = true)
 |-- country: string (nullable = true)
 |-- province: string (nullable = true)
 |-- city: string (nullable = true)
 |-- infection_case: string (nullable = true)
 |-- infected_by: string (nullable = true)
 |-- contact_number: string (nullable = true)
 |-- symptom_onset_date: string (nullable = true)
 |-- confirmed_date: string (nullable = true)
 |-- released_date: string (nullable = true)
 |-- deceased_date: string (nullable = true)
 |-- state: string (nullable = true)
 |-- no_days: double (nullable = true)
 |-- is_male: integer (nullable = false)
 |-- is_dead: integer (nullable = false)



**age** and **no_days** need to be typecasted as Double

In [22]:
df = df.withColumn("age",df.age.cast('double'))

reprint the schema to check:

In [23]:
df.printSchema()

root
 |-- patient_id: long (nullable = true)
 |-- sex: string (nullable = true)
 |-- age: double (nullable = true)
 |-- country: string (nullable = true)
 |-- province: string (nullable = true)
 |-- city: string (nullable = true)
 |-- infection_case: string (nullable = true)
 |-- infected_by: string (nullable = true)
 |-- contact_number: string (nullable = true)
 |-- symptom_onset_date: string (nullable = true)
 |-- confirmed_date: string (nullable = true)
 |-- released_date: string (nullable = true)
 |-- deceased_date: string (nullable = true)
 |-- state: string (nullable = true)
 |-- no_days: double (nullable = true)
 |-- is_male: integer (nullable = false)
 |-- is_dead: integer (nullable = false)



Kindly drop the columns below

 ["patient_id","sex","infected_by","contact_number","released_date","state","symptom_onset_date","confirmed_date","deceased_date","country","no_days","city","infection_case"]

In [24]:
cols = ("patient_id","sex","infected_by","contact_number",
        "released_date","state","symptom_onset_date","confirmed_date",
        "deceased_date","country","no_days","city","infection_case")

df = df.drop(*cols)
df.printSchema()

root
 |-- age: double (nullable = true)
 |-- province: string (nullable = true)
 |-- is_male: integer (nullable = false)
 |-- is_dead: integer (nullable = false)



Please recount the number of nulls now

In [25]:
df.select([count(when(isnan(c) | col(c).isNull(), c)).alias(c) for c in df.columns]).show()

+---+--------+-------+-------+
|age|province|is_male|is_dead|
+---+--------+-------+-------+
|  0|       0|      0|      0|
+---+--------+-------+-------+



We will handle the null values next lab using imputer, look it up if you would like.

Best of luck dears 🤩🤩🤩

If you have any questions you can reach out to me:

### Omar Hammad
#### Software Engineer
##### Email: ommar365@gmail.com
##### Phone: 01144070145
##### Linkedin: https://www.linkedin.com/in/omar-a-hammad